In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
# Definiendo fechas de inicio y final
start = datetime.datetime(1993,1,1)
end = datetime.datetime(1999,1,1)

# ACCOUNTS TABLE
      ACCOUNTS,Cuentas: cada registro describe las caracteristicas de una cuenta. 
         Columna "account_id": Identificación de la cuenta - 4500
         Columna "district_id": Identificación de distrito - 77
         Columna "frequency": Frecuencia de emisión de estados de cuenta - 3
         Columna "date": Fecha de creación de la cuenta - 1535

In [3]:
acc = pd.read_csv('account.csv', sep = ',')

In [4]:
acc.info()
acc.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   account_id   4500 non-null   int64 
 1   district_id  4500 non-null   int64 
 2   frequency    4500 non-null   object
 3   date         4500 non-null   object
dtypes: int64(2), object(2)
memory usage: 140.8+ KB


account_id     4500
district_id      77
frequency         3
date           1535
dtype: int64

In [5]:
# Se tradujeron los datos de la columna 'frecuency'
# - POPLATEK MESICNE remplazdo por MONTHLY ISSUANCE (MI) - EMISIÓN MENSUAL"
# - POPLATEK TYDNE remplazado por WEEKLY ISSUANCE (WI) - EMISIÓN SEMANAL"
# - POPLATEK PO OBRATU remplazado por ISSUANCE AFTER TRANSACTION (TI) - EMISIÓN DESPUES DE TRANSACCION"
      
acc['frequency'] = acc['frequency'].apply(lambda x: x.replace("POPLATEK MESICNE"," MONTHLY_ISSUANCE_(MI)"))
acc['frequency'] = acc['frequency'].apply(lambda x: x.replace("POPLATEK TYDNE","WEEKLY_ISSUANCE_(WI)"))
acc['frequency'] = acc['frequency'].apply(lambda x: x.replace("POPLATEK PO OBRATU","ISSUANCE_AFTER_TRANSACTION_(TI)"))

In [6]:
acc.to_csv('cleaned_account.csv', index = False)

# CLIENTS TABLE
    CLIENTS, Clientes: cada registro describe las caracteristicas de un cliente.
        Columna "client_id": Identificación de la cuenta - 5369 unicos
        Columna "birth_number": Fecha de nacimiento - 4738 unicos
        Columna "district_id": Frecuencia de emisión de estados de cuenta - 77 unicos
        Columna "sex": Genero - 2

In [7]:
client = pd.read_csv('client.csv', sep = ',')
client.head()

,client_id,birth_number,district_id,sex
0,1,13/12/1970,18,F
1,2,04/02/1945,1,M
2,3,09/10/1940,1,F
3,4,01/12/1956,5,M
4,5,03/07/1960,5,F


In [8]:
client.info()
client.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5369 entries, 0 to 5368
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   client_id     5369 non-null   int64 
 1   birth_number  5369 non-null   object
 2   district_id   5369 non-null   int64 
 3   sex           5369 non-null   object
dtypes: int64(2), object(2)
memory usage: 167.9+ KB


client_id       5369
birth_number    4738
district_id       77
sex                2
dtype: int64

In [9]:
# Modificacion del formato fecha
client['birth_number'] = client['birth_number'].astype('datetime64[ns]')

In [10]:
# obtener la edad de los clientes
def age(x):
    d = end - x
    return d.days//365

client['age'] = client['birth_number'].map(age)
del client['birth_number']
client['age_group'] = client['age'] // 10 * 10

In [11]:
client.head()

,client_id,district_id,sex,age,age_group
0,1,18,F,28,20
1,2,1,M,53,50
2,3,1,F,58,50
3,4,5,M,43,40
4,5,5,F,38,30


In [12]:
client.to_csv("cleaned_client.csv", index=False)

# DISPOSITON TABLE
    DISPOSITON (DISP), Disposición (Disp): cada registro relaciona un cliente con una sola cuenta, es decir, esta relación
    describe los derechos de los clientes para operar cuentas.
       Columna "disp_id": Identificación de registro - 5369 unicos
       Columna "client_id": Identificación de cliente - 5369 unicos
       Columna "account_id": Identificador de cuenta - 4500 unicos
       Columna "type": Tipo de disposicion, Dueno / Usuario - 2 unicos

In [13]:
disp = pd.read_csv('disp.csv', sep = ';')
disp.head()

,disp_id,client_id,account_id,type
0,1,1,1,OWNER
1,2,2,2,OWNER
2,3,3,2,DISPONENT
3,4,4,3,OWNER
4,5,5,3,DISPONENT


In [14]:
disp.info()
disp.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5369 entries, 0 to 5368
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   disp_id     5369 non-null   int64 
 1   client_id   5369 non-null   int64 
 2   account_id  5369 non-null   int64 
 3   type        5369 non-null   object
dtypes: int64(3), object(1)
memory usage: 167.9+ KB


disp_id       5369
client_id     5369
account_id    4500
type             2
dtype: int64

# ORDERS TABLE
    PERMANENT ORDERS, Debits only (ORDER), Pedidos Permanentes, solo debitos (PEDIDOS): cada registro describe las
    caracteristicas de una orden de pago.
        Columna "order_id": Identificación de registro - 6471 unicos
        Columna "account_id": Identificacion de orden de cuenta emitida para - 3758 unicos
        Columna "bank_to": Banco destinatario - 13 unicos
        Columna "account_to": Cuenta del destinatario - 6646 unicos
        Columna "amount": Importe del pedido - 4412 unicos
        Columna "k_symbol": Caracterizacion del pago - 5. 

In [15]:
order = pd.read_csv('order.csv', sep = ';')
order.head()

,order_id,account_id,bank_to,account_to,amount,k_symbol
0,29401,1,YZ,87144583,2452.0,SIPO
1,29402,2,ST,89597016,3372.7,UVER
2,29403,2,QR,13943797,7266.0,SIPO
3,29404,3,WX,83084338,1135.0,SIPO
4,29405,3,CD,24485939,327.0,


In [16]:
order.info()
order.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6471 entries, 0 to 6470
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   order_id    6471 non-null   int64  
 1   account_id  6471 non-null   int64  
 2   bank_to     6471 non-null   object 
 3   account_to  6471 non-null   int64  
 4   amount      6471 non-null   float64
 5   k_symbol    6471 non-null   object 
dtypes: float64(1), int64(3), object(2)
memory usage: 303.5+ KB


order_id      6471
account_id    3758
bank_to         13
account_to    6446
amount        4412
k_symbol         5
dtype: int64

In [17]:
# Función para traducir el significado de los 'K symbols'
def translate_k_symbol(x):
    if x == 'POJISTNE':
        return 'Insurance payment'
    elif x == 'SLUZBY':
        return 'Payment for statement'
    elif x == 'UROK':
        return 'Interest credited'
    elif x == 'SANKC. UROK':
        return 'Sanction interest if negative balance'
    elif x == 'SIPO':
        return 'Household'
    elif x == 'LEASING':
        return 'Leasing'
    elif x == 'DUCHOD':
        return 'Old-age pension'  
    elif x == 'UVER':
        return 'Loan payment'      
    else:
        return np.NaN

In [18]:
order['k_symbol'] = order['k_symbol'].map(translate_k_symbol)

In [19]:
order.to_csv('cleaned_order.csv', index = False)

# TRANSACTIONS TABLE
     TRANSACTIONS (TRANS), Transacciones (Trans) cada registro describe una transacción de una cuenta. 
        Columna "trans_id": Identificación de transaccion - 1056320 unicos
        Columna "account_id": Identificacion de la la cuenta con la que se realizo la transaccion - 4500 unicos
        Columna "date": Fecha de la transaccion - 2191 unicos
        Columna "type": +/- transaccion - 3 unicos
        Columna "operation": Modo de trasaccion - 5 unicos
        Columna "amount": Monto de la transaccion - 40400  
        Columna "balance": Balance de la cuenta despues de la transaccion - 542739
        Columna "k_symbol": Caracteristicas de la transaccion - 8
        Columna "bank": Banco del socio - 13
        Columna "account": Cuenta del socio -7665

In [20]:
trans = pd.read_csv('trans.csv', sep = ';', low_memory = False)
trans.head()

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN
3,1117247,3818,930101,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN
4,579373,1972,930102,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN


In [21]:
trans.info()
trans.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1056320 entries, 0 to 1056319
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   trans_id    1056320 non-null  int64  
 1   account_id  1056320 non-null  int64  
 2   date        1056320 non-null  int64  
 3   type        1056320 non-null  object 
 4   operation   873206 non-null   object 
 5   amount      1056320 non-null  float64
 6   balance     1056320 non-null  float64
 7   k_symbol    574439 non-null   object 
 8   bank        273508 non-null   object 
 9   account     295389 non-null   float64
dtypes: float64(3), int64(3), object(4)
memory usage: 80.6+ MB


trans_id      1056320
account_id       4500
date             2191
type                3
operation           5
amount          40400
balance        542739
k_symbol            8
bank               13
account          7665
dtype: int64

In [22]:
# Función para traducir el tipo de transacción
def translate_trans_type(x):
    if x == 'PRIJEM':
        return 'Credit'
    elif x == 'VYDAJ':
        return 'Withdrawal'
    else:
        return np.NaN    

In [23]:
# Función para traducir la operación
def translate_trans_operation(x):
    if x == 'VYBER KARTOU':
        return 'Credit card withdrawal'
    elif x == 'VKLAD':
        return 'Credit in cash'
    elif x == 'PREVOD Z UCTU':
        return 'Collection from another bank'
    elif x == 'VYBER':
        return 'Withdrawal in Cash'
    elif x == 'PREVOD NA UCET':
        return 'Remittance to another bank'    
    else:
        return np.NaN

In [24]:
trans['k_symbol'] = trans['k_symbol'].map(translate_k_symbol)
trans['operation'] = trans['operation'].map(translate_trans_operation)
trans['type'] = trans['type'].map(translate_trans_type)
trans['date'] = pd.to_datetime(trans['date'], format='%y%m%d')

In [25]:
trans.to_csv('cleaned_trans.csv', index = False)

# LOANS TABLE
     LOANS, Prestamos: Cada registro describe un prestamo concedido para una cuenta determinada.
        Columna "loan_id": Identificador de registro - 682 unicos
        Columna "account_id": Identificador de cuenta - 682 unicos
        Columna "date": Fecha de concesion del prestamo - 559 unicos
        Columna "amount": Importe de prestamo - 645 unicos
        Columna "duration": duracion del prestamo - 5 unicos
        Columna "payments": pagos mensuales del prestamo - 577 unicos  
        Columna "status": estado de la amortizacion del prestamo - 4 unicos
                            A: contrato terminado, sin problema
                            B: contrato terminado, prestamo no pagado
                            C: contrato en ejecucion, ok hasta ahora
                            D: contrato en ejecucion, cliente en deudado

In [26]:
loan = pd.read_csv('loan.csv', sep = ';')
loan.head()

,loan_id,account_id,date,amount,duration,payments,status
0,5314,1787,930705,96396,12,8033.0,B
1,5316,1801,930711,165960,36,4610.0,A
2,6863,9188,930728,127080,60,2118.0,A
3,5325,1843,930803,105804,36,2939.0,A
4,7240,11013,930906,274740,60,4579.0,A


In [27]:
loan.info()
loan.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   loan_id     682 non-null    int64  
 1   account_id  682 non-null    int64  
 2   date        682 non-null    int64  
 3   amount      682 non-null    int64  
 4   duration    682 non-null    int64  
 5   payments    682 non-null    float64
 6   status      682 non-null    object 
dtypes: float64(1), int64(5), object(1)
memory usage: 37.4+ KB


loan_id       682
account_id    682
date          559
amount        645
duration        5
payments      577
status          4
dtype: int64

In [28]:
loan['date'] = pd.to_datetime(loan['date'], format='%y%m%d')

In [29]:
loan.to_csv('cleaned_loan.csv', index = False)

#  CREDIT CARDS TABLE
    CREDIT CARDS (CARD),Tarjetas de Crédito (Tarjeta): Cada registro describe una tarjeta de crédito emitida a una cuenta. 
       Columna "card_id": Identificado de registro - 892 unicos
       Columna "disp_id": Identificador de disposicion - 892 unicos
       Columna "type": Tipo de tarjeta de credito - 3 unicos
                             junior
                             classic
                             gold
       Columna "issued": Fecha de emision de tarjeta - 607 unicos

In [30]:
card = pd.read_csv("card.csv", sep = ';')
card.head()

,card_id,disp_id,type,issued
0,1005,9285,classic,931107 00:00:00
1,104,588,classic,940119 00:00:00
2,747,4915,classic,940205 00:00:00
3,70,439,classic,940208 00:00:00
4,577,3687,classic,940215 00:00:00


In [31]:
card['issued_date'] = pd.to_datetime(card['issued'].str[:6], format='%y%m%d')
del card['issued']

In [32]:
card.info()
card.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   card_id      892 non-null    int64         
 1   disp_id      892 non-null    int64         
 2   type         892 non-null    object        
 3   issued_date  892 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 28.0+ KB


card_id        892
disp_id        892
type             3
issued_date    607
dtype: int64

In [33]:
card.head()

,card_id,disp_id,type,issued_date
0,1005,9285,classic,1993-11-07
1,104,588,classic,1994-01-19
2,747,4915,classic,1994-02-05
3,70,439,classic,1994-02-08
4,577,3687,classic,1994-02-15


In [34]:
card.to_csv("cleaned_card.csv", index=False)

# DISTRICT TABLE
    DEMOGRAPHIC DATA (DISTRICT), Datos Demográficos (Distrito): Cada registro describe las características demográficas 
    de un distrito.
        Columna "A1": Identificado de distrito - 77 unicos
        Columna "A2": Nombre del distrito - 77 unicos
        Columna "A3": Nombre de la region - 8 unicos
        Columna "A4": Numero de habitantes - 77 unicos
        Columna "A5": Numero de municipios con <499 habitantes - 53 unicos
        Columna "A6": Numero de municipios con 500-1499 habitantes - 36 unicos
        Columna "A7": Numero de municipios con 2000-9999 habitantes - 17 unicos
        Columna "A8": Numero de municipios con >10000 habitantes - 6 unicos
        Columna "A9": Numero de ciudades - 11 unicos
        Columna "A10": Proporcion de habitantes urbanos - 70 unicos
        Columna "A11": Salario medio - 76 unicos
        Columna "A12": Tasa de desempleo 1995 - 71 unicos
        Columna "A13": Tasa de desempleo de 1996 - 73 unicos
        Columna "A14": Emprendedores por cada 1000 habitantes - 44 unicos
        Columna "A15": Numero de delitos cometidos en 1995 - 76 unicos
        Columna "A16": Numero de delitos cometidos en 1996 - 76 unicos

In [35]:
dist = pd.read_csv('district.csv', sep = ';')
dist.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.67,1.85,132,2159,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,2824,2813
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,5244,5892
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.85,4.43,118,2616,3040


In [36]:
dist.rename(columns = {'A1':'district_id', 'A2':'district_name', 'A3':'district_area', 'A4':'population', 'A9':'total_of_cities', 'A11':'avg_salary'}, inplace = True)
dist.head()

,district_id,district_name,district_area,population,A5,A6,A7,A8,total_of_cities,A10,avg_salary,A12,A13,A14,A15,A16
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.67,1.85,132,2159,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,2824,2813
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,5244,5892
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.85,4.43,118,2616,3040


In [37]:
np.unique(dist['district_area'])

array(['Prague', 'central Bohemia', 'east Bohemia', 'north Bohemia',
       'north Moravia', 'south Bohemia', 'south Moravia', 'west Bohemia'],
      dtype=object)

In [38]:
dist.info()
dist.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   district_id      77 non-null     int64  
 1   district_name    77 non-null     object 
 2   district_area    77 non-null     object 
 3   population       77 non-null     int64  
 4   A5               77 non-null     int64  
 5   A6               77 non-null     int64  
 6   A7               77 non-null     int64  
 7   A8               77 non-null     int64  
 8   total_of_cities  77 non-null     int64  
 9   A10              77 non-null     float64
 10  avg_salary       77 non-null     int64  
 11  A12              77 non-null     object 
 12  A13              77 non-null     float64
 13  A14              77 non-null     int64  
 14  A15              77 non-null     object 
 15  A16              77 non-null     int64  
dtypes: float64(2), int64(10), object(4)
memory usage: 9.8+ KB


district_id        77
district_name      77
district_area       8
population         77
A5                 53
A6                 36
A7                 17
A8                  6
total_of_cities    11
A10                70
avg_salary         76
A12                71
A13                73
A14                44
A15                76
A16                76
dtype: int64

# Final de la limpieza
# - - - - - - - - - - - - - - -
# JOINS

## Creación de tabla de pagos de préstamo por categoría

In [39]:
trans_j = trans[["account_id", "amount", "balance", "k_symbol"]]
trans_j.columns = ["account_id", "trans_amount", "balance", "trans_k_symbol"]
trans_j = trans_j[trans_j['trans_k_symbol'] == 'Loan payment']
trans_j = trans_j.sort_values(by = 'account_id')
trans_j = trans_j.groupby(by = "account_id").sum()
trans_j = trans_j.reset_index()
trans_j.head()

,account_id,trans_amount,balance
0,2,80944.8,947492.1
1,19,20185.6,172066.1
2,25,30278.4,448955.3
3,37,5307.5,6140.0
4,38,18454.4,328602.1


In [40]:
disp2 = disp[disp['type']=='OWNER']

In [41]:
client_acc = pd.merge(client, disp2[['client_id','account_id','type']], on='client_id', how='left')
client_loan = pd.merge(client_acc, loan, on='account_id', how='inner')
client_loan.head().sort_values(by = 'client_id')

,client_id,district_id,sex,age,age_group,account_id,type,loan_id,date,amount,duration,payments,status
0,2,1,M,53,50,2.0,OWNER,4959,1994-01-05,80952,24,3373.0,A
1,25,21,F,59,50,19.0,OWNER,4961,1996-04-29,30276,12,2523.0,B
2,31,68,M,36,30,25.0,OWNER,4962,1997-12-08,30276,12,2523.0,A
3,45,20,M,46,40,37.0,OWNER,4967,1998-10-14,318480,60,5308.0,D
4,46,19,F,58,50,38.0,OWNER,4968,1998-04-19,110736,48,2307.0,C


In [42]:
client_trans_p = pd.merge(client_loan,trans_j, on = 'account_id', how = 'inner')
client_trans_p.head()

,client_id,district_id,sex,age,age_group,account_id,type,loan_id,date,amount,duration,payments,status,trans_amount,balance
0,2,1,M,53,50,2.0,OWNER,4959,1994-01-05,80952,24,3373.0,A,80944.8,947492.1
1,25,21,F,59,50,19.0,OWNER,4961,1996-04-29,30276,12,2523.0,B,20185.6,172066.1
2,31,68,M,36,30,25.0,OWNER,4962,1997-12-08,30276,12,2523.0,A,30278.4,448955.3
3,45,20,M,46,40,37.0,OWNER,4967,1998-10-14,318480,60,5308.0,D,5307.5,6140.0
4,46,19,F,58,50,38.0,OWNER,4968,1998-04-19,110736,48,2307.0,C,18454.4,328602.1


In [43]:
client_dist_p = pd.merge(client_trans_p,dist, on = 'district_id', how = 'inner')
client_dist_p = client_dist_p.sort_values(by = 'client_id')
client_dist_p = client_dist_p.reset_index()
client_dist_p.head()

,index,client_id,district_id,sex,age,age_group,account_id,type,loan_id,date,...,A7,A8,total_of_cities,A10,avg_salary,A12,A13,A14,A15,A16
0,0,2,1,M,53,50,2.0,OWNER,4959,1994-01-05,...,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
1,79,25,21,F,59,50,19.0,OWNER,4961,1996-04-29,...,7,1,7,67.0,9104,1.51,2.07,123,2299,2354
2,87,31,68,M,36,30,25.0,OWNER,4962,1997-12-08,...,18,2,6,57.2,9893,4.09,4.72,96,5623,5887
3,102,45,20,M,46,40,37.0,OWNER,4967,1998-10-14,...,3,1,4,58.4,8547,2.65,3.64,120,1563,1542
4,107,46,19,F,58,50,38.0,OWNER,4968,1998-04-19,...,3,1,4,52.7,8402,3.13,3.98,120,999,1099


In [44]:
client_dist_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 31 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   index            682 non-null    int64         
 1   client_id        682 non-null    int64         
 2   district_id      682 non-null    int64         
 3   sex              682 non-null    object        
 4   age              682 non-null    int64         
 5   age_group        682 non-null    int64         
 6   account_id       682 non-null    float64       
 7   type             682 non-null    object        
 8   loan_id          682 non-null    int64         
 9   date             682 non-null    datetime64[ns]
 10  amount           682 non-null    int64         
 11  duration         682 non-null    int64         
 12  payments         682 non-null    float64       
 13  status           682 non-null    object        
 14  trans_amount     682 non-null    float64  

In [45]:
client_dist_p.to_csv('Full loan payment table information.csv', index = False)

## Creación de tabla con información completa

In [46]:
trans_t = trans[["account_id", "amount", "balance", "k_symbol"]]
trans_t.columns = ["account_id", "trans_amount", "balance", "trans_k_symbol"]
trans_t = trans_t.sort_values(by = 'account_id')
trans_t = trans_t.groupby(by = "account_id").sum()
trans_t = trans_t.reset_index()
trans_t.head()

,account_id,trans_amount,balance
0,1,375174.5,3822568.0
1,2,3151479.3,17466456.7
2,3,295021.8,3155900.3
3,4,350971.4,4046141.0
4,5,166881.4,2107565.8


In [47]:
client_trans = pd.merge(client_loan,trans_t, on = 'account_id', how = 'inner')
client_trans.head()

,client_id,district_id,sex,age,age_group,account_id,type,loan_id,date,amount,duration,payments,status,trans_amount,balance
0,2,1,M,53,50,2.0,OWNER,4959,1994-01-05,80952,24,3373.0,A,3151479.3,17466456.7
1,25,21,F,59,50,19.0,OWNER,4961,1996-04-29,30276,12,2523.0,B,1575515.9,4924680.2
2,31,68,M,36,30,25.0,OWNER,4962,1997-12-08,30276,12,2523.0,A,2958545.1,15420481.2
3,45,20,M,46,40,37.0,OWNER,4967,1998-10-14,318480,60,5308.0,D,948153.9,4881164.9
4,46,19,F,58,50,38.0,OWNER,4968,1998-04-19,110736,48,2307.0,C,571949.7,4480089.7


In [48]:
client_dist = pd.merge(client_trans,dist, on = 'district_id', how = 'inner')
client_dist = client_dist.sort_values(by = 'client_id')
client_dist = client_dist.reset_index()
client_dist.head()

,index,client_id,district_id,sex,age,age_group,account_id,type,loan_id,date,...,A7,A8,total_of_cities,A10,avg_salary,A12,A13,A14,A15,A16
0,0,2,1,M,53,50,2.0,OWNER,4959,1994-01-05,...,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
1,79,25,21,F,59,50,19.0,OWNER,4961,1996-04-29,...,7,1,7,67.0,9104,1.51,2.07,123,2299,2354
2,87,31,68,M,36,30,25.0,OWNER,4962,1997-12-08,...,18,2,6,57.2,9893,4.09,4.72,96,5623,5887
3,102,45,20,M,46,40,37.0,OWNER,4967,1998-10-14,...,3,1,4,58.4,8547,2.65,3.64,120,1563,1542
4,107,46,19,F,58,50,38.0,OWNER,4968,1998-04-19,...,3,1,4,52.7,8402,3.13,3.98,120,999,1099


In [49]:
client_dist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 31 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   index            682 non-null    int64         
 1   client_id        682 non-null    int64         
 2   district_id      682 non-null    int64         
 3   sex              682 non-null    object        
 4   age              682 non-null    int64         
 5   age_group        682 non-null    int64         
 6   account_id       682 non-null    float64       
 7   type             682 non-null    object        
 8   loan_id          682 non-null    int64         
 9   date             682 non-null    datetime64[ns]
 10  amount           682 non-null    int64         
 11  duration         682 non-null    int64         
 12  payments         682 non-null    float64       
 13  status           682 non-null    object        
 14  trans_amount     682 non-null    float64  

In [50]:
client_dist.to_csv("Full table information.csv", index = False)

## Creating training DataFrame: strings to ints

In [51]:
# Función para cambiar la inicial del sexo a número
def transform_sex_to_number(x):
    if x == 'F':
        return 1
    else:
        return 2

In [52]:
# obtener el número de días desde que el préstamo se otorgó
def dayz(x):
    d = end - x
    return d.days

In [53]:
# Función para cambiar el área del distrito a número
def transform_district_area(x):
    if x == 'Prague':
        return 1
    elif x == 'central Bohemia':
        return 2
    elif x == 'east Bohemia':
        return 3
    elif x == 'north Bohemia':
        return 4
    elif x == 'north Moravia':
        return 5
    elif x == 'south Bohemia':
        return 6
    elif x == 'south Moravia':
        return 7
    elif x == 'west Bohemia':
        return 8     
    else:
        return np.NaN

In [54]:
# No sé porque la clasificación del sexo no lo hace de forma correcta 
#client_dist['sex_number'] = client['sex'].map(transform_sex_to_number)
client_dist['sex_number'] = client_dist['sex'].apply(lambda x: x.replace("M","1") if x == "M" else x.replace("F","2"))
client_dist['loan_days'] = client_dist['date'].map(dayz)
client_dist['district_a_id'] = client_dist['district_area'].map(transform_district_area)

In [55]:
del client_dist['sex']
del client_dist['type']
del client_dist['date']
del client_dist['district_name']
del client_dist['district_area']
del client_dist['A12']
del client_dist['A15']

In [56]:
client_dist.head()

,index,client_id,district_id,age,age_group,account_id,loan_id,amount,duration,payments,...,A8,total_of_cities,A10,avg_salary,A13,A14,A16,sex_number,loan_days,district_a_id
0,0,2,1,53,50,2.0,4959,80952,24,3373.0,...,1,1,100.0,12541,0.43,167,99107,1,1822,1
1,79,25,21,59,50,19.0,4961,30276,12,2523.0,...,1,7,67.0,9104,2.07,123,2354,2,977,6
2,87,31,68,36,30,25.0,4962,30276,12,2523.0,...,2,6,57.2,9893,4.72,96,5887,1,389,5
3,102,45,20,46,40,37.0,4967,318480,60,5308.0,...,1,4,58.4,8547,3.64,120,1542,1,79,6
4,107,46,19,58,50,38.0,4968,110736,48,2307.0,...,1,4,52.7,8402,3.98,120,1099,2,257,6


In [57]:
client_dist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   index            682 non-null    int64  
 1   client_id        682 non-null    int64  
 2   district_id      682 non-null    int64  
 3   age              682 non-null    int64  
 4   age_group        682 non-null    int64  
 5   account_id       682 non-null    float64
 6   loan_id          682 non-null    int64  
 7   amount           682 non-null    int64  
 8   duration         682 non-null    int64  
 9   payments         682 non-null    float64
 10  status           682 non-null    object 
 11  trans_amount     682 non-null    float64
 12  balance          682 non-null    float64
 13  population       682 non-null    int64  
 14  A5               682 non-null    int64  
 15  A6               682 non-null    int64  
 16  A7               682 non-null    int64  
 17  A8              

In [58]:
client_dist['sex_number'] = client_dist['sex_number'].astype(int)

In [59]:
client_dist.to_csv("Training DataFrame.csv", index = False)